# Stanza: A Tutorial on the Python CoreNLP Interface

![Latest Version](https://img.shields.io/pypi/v/stanza.svg?colorB=bc4545)
![Python Versions](https://img.shields.io/pypi/pyversions/stanza.svg?colorB=bc4545)

While the Stanza library implements accurate neural network modules for basic functionalities such as part-of-speech tagging and dependency parsing, the [Stanford CoreNLP Java library](https://stanfordnlp.github.io/CoreNLP/) has been developed for years and offers more complementary features such as coreference resolution and relation extraction. To unlock these features, the Stanza library also offers an officially maintained Python interface to the CoreNLP Java library. This interface allows you to get NLP anntotations from CoreNLP by writing native Python code.


This tutorial walks you through the installation, setup and basic usage of this Python CoreNLP interface. If you want to learn how to use the neural network components in Stanza, please refer to other tutorials.

## 1. Installation

Before the installation starts, please make sure that you have Python 3 and Java installed on your computer. Since Colab already has them installed, we'll skip this procedure in this notebook.

### Installing Stanza

Installing and importing Stanza are as simple as running the following commands:

In [1]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Setting up Stanford CoreNLP

In order for the interface to work, the Stanford CoreNLP library has to be installed and a `CORENLP_HOME` environment variable has to be pointed to the installation location.

Here we are going to show you how to download and install the CoreNLP library on your machine, with Stanza's installation command:

In [2]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

That's all for the installation! 🎉  We can now double check if the installation is successful by listing files in the CoreNLP directory. You should be able to see a number of `.jar` files by running the following command:

In [3]:
# Examine the CoreNLP installation folder to make sure the installation is successful
!ls $CORENLP_HOME

build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.2.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.1.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.1-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.1-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.1-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

**Note 1**:
If you are want to use the interface in a terminal (instead of a Colab notebook), you can properly set the `CORENLP_HOME` environment variable with:

```bash
export CORENLP_HOME=path_to_corenlp_dir
```

Here we instead set this variable with the Python `os` library, simply because `export` command is not well-supported in Colab notebook.


**Note 2**:
The `stanza.install_corenlp()` function is only available since Stanza v1.1.1. If you are using an earlier version of Stanza, please check out our [manual installation page](https://stanfordnlp.github.io/stanza/client_setup.html#manual-installation) for how to install CoreNLP on your computer.

**Note 3**:
Besides the installation function, we also provide a `stanza.download_corenlp_models()` function to help you download additional CoreNLP models for different languages that are not shipped with the default installation. Check out our [automatic installation website page](https://stanfordnlp.github.io/stanza/client_setup.html#automated-installation) for more information on how to use it.

## 2. Annotating Text with CoreNLP Interface

### Constructing CoreNLPClient

At a high level, the CoreNLP Python interface works by first starting a background Java CoreNLP server process, and then initializing a client instance in Python which can pass the text to the background server process, and accept the returned annotation results.

We wrap these functionalities in a `CoreNLPClient` class. Therefore, we need to start by importing this class from Stanza.

In [4]:
# Import client module
from stanza.server import CoreNLPClient

After the import is done, we can construct a `CoreNLPClient` instance. The constructor method takes a Python list of annotator names as argument. Here let's explore some basic annotators including tokenization, sentence split, part-of-speech tagging, lemmatization and named entity recognition (NER). 

Additionally, the client constructor accepts a `memory` argument, which specifies how much memory will be allocated to the background Java process. An `endpoint` option can be used to specify a port number used by the communication between the server and the client. The default port is 9000. However, since this port is pre-occupied by a system process in Colab, we'll manually set it to 9001 in the following example.

Also, here we manually set `be_quiet=True` to avoid an IO issue in colab notebook. You should be able to use `be_quiet=False` on your own computer, which will print detailed logging information from CoreNLP during usage.

For more options in constructing the clients, please refer to the [CoreNLP Client Options List](https://stanfordnlp.github.io/stanza/corenlp_client.html#corenlp-client-options).

In [5]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-cba3397000f543bf.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-cba3397000f543bf.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized


In [40]:
from collections import defaultdict
from stanza.server import CoreNLPClient

#text = "Barack Obama was born in Hawaii. In 2008 he became the president."
text = '''Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson about the sale, but the latter was too inebriated to notice.'''
doc = client.annotate(text)

animacy = defaultdict(dict)
print(doc.corefChain[0])
#for x in doc.corefChain:
 # print(x)
i = 2
for s in doc.sentence[0:7]:
  print('SENTENCE ', i)
  i+=1
  for m in s.mentions:
    print(m)


chainID: 44
mention {
  mentionID: 19
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 15
  endIndex: 16
  headIndex: 15
  sentenceIndex: 2
  position: 7
}
mention {
  mentionID: 27
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 18
  endIndex: 19
  headIndex: 18
  sentenceIndex: 3
  position: 6
}
mention {
  mentionID: 33
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 2
  endIndex: 3
  headIndex: 2
  sentenceIndex: 4
  position: 2
}
mention {
  mentionID: 44
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 2
  endIndex: 3
  headIndex: 2
  sentenceIndex: 6
  position: 2
}
representative: 0

SENTENCE  2
sentenceIndex: 0
tokenStartInSentenceInclusive: 0
tokenEndInSentenceExclusive: 2
ner: "PERSON"
entityType: "PERSON"
entityMentionIndex: 0
canonicalEntityMentionIndex: 0
entity

In [17]:
print(doc.sentence.mention)

AttributeError: ignored

In [14]:
from collections import defaultdict
from stanza.server import CoreNLPClient

text = "Barack Obama was born in Hawaii. In 2008 he became the president."

doc = client.annotate(text)

animacy = defaultdict(dict)
for x in doc.corefChain:
    for y in x.mention:
        print(y.animacy)
        for i in range(y.beginIndex, y.endIndex):
            animacy[y.sentenceIndex][i] = True
            print(y.sentenceIndex, i)

for sent_idx, sent in enumerate(doc.sentence):
    print("[Sentence {}]".format(sent_idx+1))
    for t_idx, token in enumerate(sent.token):
        animate = animacy[sent_idx].get(t_idx, False)
        print("{:12s}\t{:12s}\t{:6s}\t{:20s}\t{}".format(token.word, token.lemma, token.pos, token.ner, animate))
    print("")

ANIMATE
0 0
0 1
ANIMATE
1 2
[Sentence 1]
Barack      	Barack      	NNP   	PERSON              	True
Obama       	Obama       	NNP   	PERSON              	True
was         	be          	VBD   	O                   	False
born        	bear        	VBN   	O                   	False
in          	in          	IN    	O                   	False
Hawaii      	Hawaii      	NNP   	STATE_OR_PROVINCE   	False
.           	.           	.     	O                   	False

[Sentence 2]
In          	in          	IN    	O                   	False
2008        	2008        	CD    	DATE                	False
he          	he          	PRP   	O                   	True
became      	become      	VBD   	O                   	False
the         	the         	DT    	O                   	False
president   	president   	NN    	TITLE               	False
.           	.           	.     	O                   	False



In [66]:
text = '''Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson about the sale, but the latter was too inebriated to notice.'''
doc2 = client.annotate(text)
chains = doc2.corefChain
chain_dict=dict()
for index_chain,chain in enumerate(chains):
    chain_dict[index_chain]={}
    chain_dict[index_chain]['ref']=''
    chain_dict[index_chain]['mentions']=[{'mentionID':mention.mentionID,
                                          'mentionType':mention.mentionType,
                                          'number':mention.number,
                                          'gender':mention.gender,
                                          'animacy':mention.animacy,
                                          'beginIndex':mention.beginIndex,
                                          'endIndex':mention.endIndex,
                                          'headIndex':mention.headIndex,
                                          'sentenceIndex':mention.sentenceIndex,
                                          'position':mention.position,
                                          'ref':'',
                                          } for mention in chain.mention ]


for k,v in chain_dict.items():
    print('key',k)
    mentions=v['mentions']
    for mention in mentions:
        words_list = doc2.sentence[mention['sentenceIndex']].token[mention['beginIndex']:mention['endIndex']]
        for t in words_list:
          if (t.pos == 'NNP' or t.pos == 'PRP'):
            mention['ref']=t.word
            print(mention['ref'],k, mention['sentenceIndex'], mention['beginIndex'])  
        

key 0
Ally 0 2 15
Ally 0 3 18
Ally 0 4 2
Ally 0 6 2
key 1
Bobby 1 7 24
Bobby 1 8 7
Bobby 1 9 4
key 2
him 2 6 8
Jackson 2 3 0
she 2 5 4
Jackson 2 7 5
Jackson 2 5 9
he 2 9 7
Jackson 2 7 10
Jackson 2 9 10
Jackson 2 4 0
Jackson 2 2 4
him 2 5 17
Jackson 2 6 0
him 2 3 21
she 2 3 24
he 2 2 10
Jackson 2 8 5


After the above code block finishes executing, if you print the background processes, you should be able to find the Java CoreNLP server running.

In [6]:
# Print background processes and look for java
# You should be able to see a StanfordCoreNLPServer java process running in the background
!ps -o pid,cmd | grep java

    138 java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-e9c4bcaa73fb4783.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized
    159 /bin/bash -c ps -o pid,cmd | grep java
    161 grep java


### Annotating Text

Annotating a piece of text is as simple as passing the text into an `annotate` function of the client object. After the annotation is complete, a `Document`  object will be returned with all annotations.

Note that although in general annotations are very fast, the first annotation might take a while to complete in the notebook. Please stay patient.

In [7]:
# Annotate some text
text = '''Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson about the sale, but the latter was too inebriated to notice.'''
document = client.annotate(text)
print(type(document))

<class 'CoreNLP_pb2.Document'>


## 3. Accessing Annotations

Annotations can be accessed from the returned `Document` object.

A `Document` contains a list of `Sentence`s, which contain a list of `Token`s. Here let's first explore the annotations stored in all tokens.

In [8]:
# Iterate over all tokens in all sentences, and print out the word, lemma, pos and ner tags
print("{:12s}\t{:12s}\t{:6s}\t{}".format("Word", "Lemma", "POS", "NER"))

for i, sent in enumerate(document.sentence):
    print("[Sentence {}]".format(i+1))
    for t in sent.token:
        print("{:12s}\t{:12s}\t{:6s}\t{}".format(t.word, t.lemma, t.pos, t.ner))
    print("")

Word        	Lemma       	POS   	NER
[Sentence 1]
Jackson     	Jackson     	NNP   	PERSON
Maine       	Maine       	NNP   	PERSON
,           	,           	,     	O
a           	a           	DT    	O
famous      	famous      	JJ    	O
country     	country     	NN    	O
music       	music       	NN    	O
singer      	singer      	NN    	TITLE
privately   	privately   	RB    	O
battling    	battle      	VBG   	O
an          	a           	DT    	O
alcohol     	alcohol     	NN    	O
and         	and         	CC    	O
drug        	drug        	NN    	O
addiction   	addiction   	NN    	O
,           	,           	,     	O
plays       	play        	VBZ   	O
a           	a           	DT    	O
concert     	concert     	NN    	O
in          	in          	IN    	O
California  	California  	NNP   	STATE_OR_PROVINCE
.           	.           	.     	O

[Sentence 2]
His         	he          	PRP$  	O
main        	main        	JJ    	O
support     	support     	NN    	O
is          	be          	VBZ  

Alternatively, you can also browse the NER results by iterating over entity mentions over the sentences. For example:

In [19]:
# get an entity mention from the first sentence
print(document.sentence[0].mentions[0].entityMentionText)

# access the coref chain in the input text
print(document.corefChain)

Jackson Maine
[chainID: 44
mention {
  mentionID: 19
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 15
  endIndex: 16
  headIndex: 15
  sentenceIndex: 2
  position: 7
}
mention {
  mentionID: 27
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 18
  endIndex: 19
  headIndex: 18
  sentenceIndex: 3
  position: 6
}
mention {
  mentionID: 33
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 2
  endIndex: 3
  headIndex: 2
  sentenceIndex: 4
  position: 2
}
mention {
  mentionID: 44
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "NEUTRAL"
  animacy: "INANIMATE"
  beginIndex: 2
  endIndex: 3
  headIndex: 2
  sentenceIndex: 6
  position: 2
}
representative: 0
, chainID: 61
mention {
  mentionID: 52
  mentionType: "PROPER"
  number: "SINGULAR"
  gender: "MALE"
  animacy: "ANIMATE"
  beginIndex: 24
  endIndex: 25
  headIndex: 24
  

In [9]:
# Iterate over all detected entity mentions
print("{:30s}\t{}".format("Mention", "Type"))

for sent in document.sentence:
    for m in sent.mentions:
        print("{:30s}\t{}".format(m.entityMentionText, m.entityType))

Mention                       	Type
Jackson Maine                 	PERSON
singer                        	TITLE
California                    	STATE_OR_PROVINCE
Bobby                         	PERSON
manager                       	TITLE
His                           	PERSON
his                           	PERSON
Jackson                       	PERSON
waitress                      	TITLE
singer-songwriter             	TITLE
he                            	PERSON
Jackson                       	PERSON
night                         	TIME
her                           	PERSON
him                           	PERSON
she                           	PERSON
Jackson                       	PERSON
his                           	PERSON
Jackson                       	PERSON
her                           	PERSON
she                           	PERSON
him                           	PERSON
Jackson                       	PERSON
him                           	PERSON
Arizona                       	STATE_OR_PROVINC

To print all annotations a sentence, token or mention has, you can simply print the corresponding obejct.

In [13]:
print(text)

Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson 

In [16]:
# Print annotations of a token
#print(document.sentence[0].token[0])

# Print annotations of a mention
for s in document.sentence[0:1]:
  for m in s.token:
    print(m)

word: "Jackson"
pos: "NNP"
value: "Jackson"
before: ""
after: " "
originalText: "Jackson"
ner: "PERSON"
lemma: "Jackson"
beginChar: 0
endChar: 7
utterance: 0
speaker: "PER0"
beginIndex: 0
endIndex: 1
tokenBeginIndex: 0
tokenEndIndex: 1
hasXmlContext: false
isNewline: false
coarseNER: "PERSON"
fineGrainedNER: "PERSON"
corefMentionIndex: 1
entityMentionIndex: 0
nerLabelProbs: "PERSON=0.6487090475764525"

word: "Maine"
pos: "NNP"
value: "Maine"
before: " "
after: ""
originalText: "Maine"
ner: "PERSON"
lemma: "Maine"
beginChar: 8
endChar: 13
utterance: 0
speaker: "PER0"
beginIndex: 1
endIndex: 2
tokenBeginIndex: 1
tokenEndIndex: 2
hasXmlContext: false
isNewline: false
coarseNER: "PERSON"
fineGrainedNER: "PERSON"
corefMentionIndex: 1
entityMentionIndex: 0
nerLabelProbs: "PERSON=0.5448632685349827"

word: ","
pos: ","
value: ","
before: ""
after: " "
originalText: ","
ner: "O"
lemma: ","
beginChar: 13
endChar: 14
utterance: 0
speaker: "PER0"
beginIndex: 2
endIndex: 3
tokenBeginIndex: 2
token

**Note**: Since the Stanza CoreNLP client interface simply ports the CoreNLP annotation results to native Python objects, for a comprehensive lists of available annotators and how their annotation results can be accessed, you will need to visit the [Stanford CoreNLP website](https://stanfordnlp.github.io/CoreNLP/).

## 4. Shutting Down the CoreNLP Server

To shut down the background CoreNLP server process, simply call the `stop` function of the client. Note that once a server is shutdown, you'll have to restart the server with the `start()` function before any annotation is requested.

In [11]:
# Shut down the background CoreNLP server
client.stop()

time.sleep(10)
!ps -o pid,cmd | grep java

    206 /bin/bash -c ps -o pid,cmd | grep java
    208 grep java


### More Information

For more information on how to use the `CoreNLPClient`, please go to the [CoreNLPClient documentation page](https://stanfordnlp.github.io/stanza/corenlp_client.html).

## 5. Simplifying Client Usage with the Python `with` statement

In the above demo, we explicitly called the `client.start()` and `client.stop()` functions to start and stop a client-server connection. However, doing this in practice is usually suboptimal, since you may forget to call the `stop()` function at the end, resulting in an unused server process occupying your machine memory.

To solve is, a simple solution is to use the client interface with the [Python `with` statement](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement). The `with` statement provides an elegant way to automatically start and stop the server process in your Python program, without you needing to worry about this. The following code snippet demonstrates how to establish a client, annotate an example text and then stop the server with a simple `with` statement. Note that we **always recommend** you to use the `with` statement when working with the Stanza CoreNLP client interface.

In [25]:
print("Starting a server with the Python \"with\" statement...")
with CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'depparse', 'coref'], 
                   memory='4G', endpoint='http://localhost:9001', be_quiet=True) as client:
    text = "Albert Einstein was a German-born theoretical physicist."
    document = client.annotate(text)

    print("{:30s}\t{}".format("Mention", "Type"))
    for sent in document.sentence:
        for m in sent.mentions:
            print("{:30s}\t{}".format(m.entityMentionText, m.entityType))

print("\nThe server should be stopped upon exit from the \"with\" statement.")

INFO:stanza:Writing properties to tmp file: corenlp_server-47a4e7ffbff44542.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-47a4e7ffbff44542.props -annotators tokenize,ssplit,pos,lemma,ner,depparse,coref -preload -outputFormat serialized


Starting a server with the Python "with" statement...
Mention                       	Type
Albert Einstein               	PERSON
German                        	NATIONALITY
theoretical physicist         	TITLE

The server should be stopped upon exit from the "with" statement.


In [35]:
print(text)

Albert Einstein was a German-born theoretical physicist.


In [36]:
for s in document.sentence:
  print(s.mentions[0])

sentenceIndex: 0
tokenStartInSentenceInclusive: 0
tokenEndInSentenceExclusive: 2
ner: "PERSON"
entityType: "PERSON"
entityMentionIndex: 0
canonicalEntityMentionIndex: 0
entityMentionText: "Albert Einstein"



In [20]:
from collections import defaultdict
from stanza.server import CoreNLPClient

client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner','parse', 'depparse', 'coref'],
    be_quiet=False)

text = "Barack Obama was born in Hawaii.  In 2008 he became the president."

doc = client.annotate(text)

animacy = defaultdict(dict)
for x in doc.corefChain:
    for y in x.mention:
        print(y.animacy)
        for i in range(y.beginIndex, y.endIndex):
            animacy[y.sentenceIndex][i] = True
            print(y.sentenceIndex, i)

for sent_idx, sent in enumerate(doc.sentence):
    print("[Sentence {}]".format(sent_idx+1))
    for t_idx, token in enumerate(sent.token):
        animate = animacy[sent_idx].get(t_idx, False)
        print("{:12s}\t{:12s}\t{:6s}\t{:20s}\t{}".format(token.word, token.lemma, token.pos, token.ner, animate))
    print("")

INFO:stanza:Writing properties to tmp file: corenlp_server-a31f64d4bb294f54.props
INFO:stanza:Starting server with command: java -Xmx5G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-a31f64d4bb294f54.props -annotators tokenize,ssplit,pos,lemma,ner,coref -preload -outputFormat serialized


PermanentlyFailedException: ignored

## 6. Other Resources

- [Stanza Homepage](https://stanfordnlp.github.io/stanza/)
- [FAQs](https://stanfordnlp.github.io/stanza/faq.html)
- [GitHub Repo](https://github.com/stanfordnlp/stanza)
- [Reporting Issues](https://github.com/stanfordnlp/stanza/issues)
